![spotify_logo](images/spotify_logo3.png)

# Spotify API Scrape

## Helpful Links:
- [Spotify Web API - Authorization Guide](https://developer.spotify.com/documentation/general/guides/authorization-guide/)
- [Spotify API References](https://developer.spotify.com/documentation/web-api/reference/)

In [66]:
import config
import os
import requests
import json
from json import JSONEncoder
import pandas as pd
from datetime import datetime,timezone, timedelta
from tqdm import tqdm
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
from urllib.parse import urlencode

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import music21
import xlsxwriter

## Step 1: Get Access Token

In [2]:
client_id = config.client_id
client_secret = config.client_secret

In [3]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

## Step 2: Pull Recently Played

In [4]:
base_url = 'https://api.spotify.com/v1/me/player/recently-played?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
personal_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"&limit=50", headers=headers)
personal_data.append(json.loads(r.text))

In [5]:
personal_data[0]['items'][1]

{'track': {'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0epOFNiUfyON9EYx7Tpr6V'},
     'href': 'https://api.spotify.com/v1/artists/0epOFNiUfyON9EYx7Tpr6V',
     'id': '0epOFNiUfyON9EYx7Tpr6V',
     'name': 'The Strokes',
     'type': 'artist',
     'uri': 'spotify:artist:0epOFNiUfyON9EYx7Tpr6V'}],
   'available_markets': ['AD',
    'AE',
    'AL',
    'AR',
    'AT',
    'AU',
    'BA',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'BY',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'GT',
    'HK',
    'HN',
    'HR',
    'HU',
    'ID',
    'IE',
    'IL',
    'IN',
    'IS',
    'IT',
    'JO',
    'JP',
    'KW',
    'KZ',
    'LB',
    'LI',
    'LT',
    'LU',
    'LV',
    'MA',
    'MC',
    'MD',
    'ME',
    'MK',
    'MT',
    'MX',
    'MY',
    'NI',
   

In [6]:
track_ids = []
album_names = []
artist_names = []
track_names = []

for i in range(len(personal_data[0]['items'])):
    track_ids.append(personal_data[0]['items'][i]['track']['id']) # Track ID
    album_names.append(personal_data[0]['items'][i]['track']['album']['name']) # Album Name
    artist_names.append(personal_data[0]['items'][i]['track']['artists'][0]['name']) # Artist Name
    track_names.append(personal_data[0]['items'][i]['track']['name']) # Track Name

In [7]:
list_dic={'track_id':track_ids,
          'track_name':track_names,
          'artist_name':artist_names,
          'album_name':album_names,
    }

In [8]:
df1=pd.DataFrame(list_dic)
df1

track_id                                 track_name  \
0   6ta5yavnnEfCE4faU0jebM                                    Mr Blue   
1   7dz48pntblPzJ9mTPiUH81                                At The Door   
2   7Cy7wa746ywgkwIOFJtSor       Yoko Ono (Stripped with Matt Maeson)   
3   4ZdmTNaBTErD8n9AQE0YaX                              British Bombs   
4   2nUV1fiD45RN6cQZ85GDc1                                Salt (Nlmg)   
5   3xMQOd1C3TXsjQ3pmzOmkC                             Weight of Love   
6   3Hx7RXqCS7Kzjy2ot2q1Gk                                      Fever   
7   2MVwrvjmcdt4MsYYLCYMt8                                 Tighten Up   
8   1PXsUXSM3LF2XNSkmIldPb                    Little Black Submarines   
9   25YlltWXRb9k7KbrEBRuhJ                                      Lo/Hi   
10  0grFc6klR3hxoHLcgCYsF4                            Howlin' for You   
11  5lN1EH25gdiqT1SFALMAq1                        Gold on the Ceiling   
12  5G1sTBGbZT5o4PNRc75RKI                                 Lonely Boy   
13  2zmDSwptUhsGSkPc4fcHTK                              Pocket Change   
14  2zmDSwptUhsGSkPc4fcHTK                              Pocket Change   
15  2zmDSwptUhsGSkPc4fcHTK                              Pocket Change   
16  1hB8pAcdgVzSLbnFMYXKOc  Battered, Black & Blue (Hole In My Heart)   
17  1yY6FYzGRzM6odwTG7RnHV                                    Radical   
18  6YA5dsZtMM68VkEpbSSpcD                       California Halo Blue   
19  2X18YLYhIFqwKXFsYAy4if                           Lightning Riders   
20  4Uu7xa2OnZsCRu5RN911na                     Mayday!!! Fiesta Fever   
21  7qVsMCptRS8tf4eIsQVHYR                        Slam (Angel Miners)   
22  1t4wa5r7E7oZ2D4G07JFsI                                   The Best   
23  1EOQNCk2P9KeUUerIB152U                                   buzz cut   
24  4NK49OnNDP3oJcTGs89XSg                              Beggar's Song   
25  0f2QFb1ghqUzM5Cl3HTQcc                                 Be My Fire   
26  03seCg7mR5SiQ9J7NkTyfc                                Boy Like Me   
27  40CASSesK4IyM8Ud5sMsdp                                  Best Part   
28  21vwWfS6qvxh8TN3lgHvZr                                       Baby   
29  4dkOXnETdDqnKJrc46bS0a             Again (Text Voter XX to 40649)   
30  5vdNoFezjTsQTrZxPK8tDg                          Golden Dandelions   
31  3GjLecyioIsSt0EaNecctC                                Underground   
32  4y2a8Gi924VQXgHdZQlXQd                                       Stay   
33  09akcdeCiV2jIl6x9WpmxA                      don't worry, you will   
34  0eUgkunDciOOiao9Sjw3PW                                    Sinners   
35  12VzUUrRwSGJmpAaFkjDcm                                      Bells   
36  0Cz3LgsBXYwe0vM42614Gn                                 Going Home   
37  0FNvByQpQYUk3eHrqcQHSE                              The Apartment   
38  6XZ8SzOsrIPce5JdgMeJFF                                   Rockstar   
39  23yJAxeadKCI0OObMaqrrp                                  Hoodie Up   
40  7noHzxuNmeIwYftylZfVRT                                Heat Seeker   
41  62ttMFG4vm8gHtp70XV3pi           Peaches (Text Voter XX to 40649)   
42  3ATD9XGdgIRLAhviHOIh8q                                   High Low   
43  5UIR8dRPP1GtdxYHYPwKN5                                  I See You   
44  3Gw2X0ZJgxlggAIuG36rVF                                     Hollow   
45  7atYf4ccJ8L5QA4dzXcQN1                                   Dopamine   
46  2t0TmblCJctqK13OyRdSmD                                       Home   
47  6YjFI4i4mUHL54T67ucGj6                                Missing You   
48  14NNdJ03Vt6QfrY99FpmE5                           Hard To Be Alone   
49  1i0DLm8xlgb6DRgVJSu3ej                                       Lucy   

                artist_name                                    album_name  
0           Catherine Feeny                               Hurricane Glass  
1               The Strokes                                   At The Door  
2                 Moby Rich                           Yoko Ono (St

## Step 3: Pull User's Top Artists & Tracks

In [9]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-top-read'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [10]:
base_url = 'https://api.spotify.com/v1/me/top/tracks?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
top_track_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"time_range=medium_term"+"&&limit=50", headers=headers)
top_track_data.append(json.loads(r.text))

In [11]:
top_track_data[0]

{'items': [{'album': {'album_type': 'ALBUM',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
      'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
      'id': '06HL4z0CvFAxyc27GXpf02',
      'name': 'Taylor Swift',
      'type': 'artist',
      'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
    'available_markets': ['AD',
     'AE',
     'AR',
     'AT',
     'AU',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
     'LB',
     'LI',
     'LT',
     'LU',
     'LV',
     'MA',
     'MC',
     'MT',
     'MX',
     'MY',
     'NI',
     'NL',
     'NO',
 

In [12]:
track_idss = []
album_namess = []
album_relase_datess = []
artist_namess = []
track_namess = []

for i in range(len(top_track_data[0]['items'])):
    track_idss.append(top_track_data[0]['items'][i]['id']) # Track ID
    album_namess.append(top_track_data[0]['items'][i]['album']['name']) # Album Name
    album_relase_datess.append(top_track_data[0]['items'][i]['album']['release_date'])
    artist_namess.append(top_track_data[0]['items'][i]['album']['artists'][0]['name']) # Artist Name
    track_namess.append(top_track_data[0]['items'][i]['name']) # Track Name

In [13]:
list_dic2={'track_id':track_idss,
           'track_name':track_namess,
           'album_name':album_namess,
           'artist_name':artist_namess,
           'album_relase_date':album_relase_datess,
    }

In [14]:
df2=pd.DataFrame(list_dic2)
df2

track_id                              track_name  \
0   4pvb0WLRcMtbPGmtejJJ6y                  exile (feat. Bon Iver)   
1   6xZ4Q2k2ompmDppyeESIY8                        Level of Concern   
2   1ci0BoqpvH73L2TJzHhw9y                        Modern Chemistry   
3   4w2tfK0JA8KrVegKnxukf4                    The Kids Are Alright   
4   0VjIjW4GlUZAMYd2vXMi3b                         Blinding Lights   
5   6xQpOC55lufcqXvSzp7GTb                        Hard To Be Alone   
6   1uddOsj7TyRA13hnS2yDyk              Can't Take My Eyes Off You   
7   2nUV1fiD45RN6cQZ85GDc1                             Salt (Nlmg)   
8   3FRJFImdfX5NSY3QH3jI4u                              Glistening   
9   3flgdcFBWI84DPi4s1jhhd                                Savannah   
10  3iRiQJsQmH0yPIOJn3Y4WQ                           Heart & Arrow   
11  4ZdmTNaBTErD8n9AQE0YaX                           British Bombs   
12  5PNAsZO4JyT8fUzPyKwZ7W                                 Poppies   
13  5vNd5a48igf00CALQgTso1                                Comme Ça   
14  0elCmyfISzkP5tAYTVuYjS                                  August   
15  1MgV7FIyNxIG7WzMRJV5HC                       my tears ricochet   
16  0ZNU020wNYvgW84iljPkPP                              mirrorball   
17  00c805RUf4tZyrlycaA2AQ                            London Girls   
18  1RgmiiAY2FVLuGcG04ah4F                    Something You Needed   
19  2n33zaHpprIqOiSyoC8I5F     Did I Love You (Like I Promised To)   
20  3hUxzQpSfdDqwM3ZTFQY0K                                  august   
21  7kt9e9LFSpN1zQtYEl19o1                       this is me trying   
22  39UQnyET00p3VpvJoEgF5P                    Something In The Air   
23  4kiOoB3NVLKrs61jTNieAr                                Fletcher   
24  7dz48pntblPzJ9mTPiUH81                             At The Door   
25  64lsIF5pw0sJY0gV5kz0RN                 Can I Call You Tonight?   
26  7Cy7wa746ywgkwIOFJtSor    Yoko Ono (Stripped with Matt Maeson)   
27  0klnCzWGjoRQBjoPYe44Gl  Here Comes the End (feat. Judith Hill)   
28  6YjFI4i4mUHL54T67ucGj6                             Missing You   
29  2Eeur20xVqfUoM3Q7EFPFt         the last great american dynasty   
30  2t0TmblCJctqK13OyRdSmD                                    Home   
31  0Jlcvv8IykzHaSmj49uNW8                                   the 1   
32  72z1OAURj2XwHbZdBg3zpV                                Castaway   
33  2NmsngXHeC1GQ9wWrzhOMf                         illicit affairs   
34  6KJqZcs9XDgVck7Lg9QOTC                                   seven   
35  2BIfG6wL1t5wk1KixoK2BV                                 Chicago   
36  7atYf4ccJ8L5QA4dzXcQN1                                Dopamine   
37  0XZV3zE3j10RPG6Zvxsb6n                                  Nickel   
38  38orSZZSkX6gxDC2enAM4k                     It's Ok to Be Human   
39  4R2kfaDFhslZEMJqAFNpdd                                cardigan   
40  5wxjGTx4Q8esdYMd7SWHZI                                  Hollow   
41  0FDtfURDoNGv0sjNnvscPR                                    "99"   
42  5G1sTBGbZT5o4PNRc75RKI                              Lonely Boy   
43  0gbFh1Mx2cTHWVnZp6IypI                              Turn It Up   
44  7qVsMCptRS8tf4eIsQVHYR                     Slam (Angel Miners)   
45  6VsvKPJ4xjVNKpI8VVZ3SV                        invisible string   
46  0xTmJJm3wSlJ1877tW8GrL                    Here Comes the Night   
47  7krfJLNZ3Q1ITV3PKVbyoM                               You And I   
48  21lfQ6cXJF15ubyPLB6qHz                                 Vanilla   
49  554dAjQxsN1DrNL3dKgp5c                    A Toast and a Spirit   

                                       album_name              artist_name  \
0                                        folklore             Taylor Swift   
1                                Level of Concern        Twenty One Pilots   
2                            Tell All Your Friend               Okey Dokey   
3                                             404           Barns Courtney   
4                                     After Hours        

## Step 4: Pull Track Info
- API Doc: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/

In [15]:
track_id_ls = df2['track_id'].tolist()

In [16]:
track_id_ls

['4pvb0WLRcMtbPGmtejJJ6y',
 '6xZ4Q2k2ompmDppyeESIY8',
 '1ci0BoqpvH73L2TJzHhw9y',
 '4w2tfK0JA8KrVegKnxukf4',
 '0VjIjW4GlUZAMYd2vXMi3b',
 '6xQpOC55lufcqXvSzp7GTb',
 '1uddOsj7TyRA13hnS2yDyk',
 '2nUV1fiD45RN6cQZ85GDc1',
 '3FRJFImdfX5NSY3QH3jI4u',
 '3flgdcFBWI84DPi4s1jhhd',
 '3iRiQJsQmH0yPIOJn3Y4WQ',
 '4ZdmTNaBTErD8n9AQE0YaX',
 '5PNAsZO4JyT8fUzPyKwZ7W',
 '5vNd5a48igf00CALQgTso1',
 '0elCmyfISzkP5tAYTVuYjS',
 '1MgV7FIyNxIG7WzMRJV5HC',
 '0ZNU020wNYvgW84iljPkPP',
 '00c805RUf4tZyrlycaA2AQ',
 '1RgmiiAY2FVLuGcG04ah4F',
 '2n33zaHpprIqOiSyoC8I5F',
 '3hUxzQpSfdDqwM3ZTFQY0K',
 '7kt9e9LFSpN1zQtYEl19o1',
 '39UQnyET00p3VpvJoEgF5P',
 '4kiOoB3NVLKrs61jTNieAr',
 '7dz48pntblPzJ9mTPiUH81',
 '64lsIF5pw0sJY0gV5kz0RN',
 '7Cy7wa746ywgkwIOFJtSor',
 '0klnCzWGjoRQBjoPYe44Gl',
 '6YjFI4i4mUHL54T67ucGj6',
 '2Eeur20xVqfUoM3Q7EFPFt',
 '2t0TmblCJctqK13OyRdSmD',
 '0Jlcvv8IykzHaSmj49uNW8',
 '72z1OAURj2XwHbZdBg3zpV',
 '2NmsngXHeC1GQ9wWrzhOMf',
 '6KJqZcs9XDgVck7Lg9QOTC',
 '2BIfG6wL1t5wk1KixoK2BV',
 '7atYf4ccJ8L5QA4dzXcQN1',
 

In [17]:
track_data = []
for id in tqdm(track_id_ls):
    base_url = f'https://api.spotify.com/v1/audio-features/{id}?'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    r = requests.get(base_url, headers=headers)
    track_data.append(json.loads(r.text))

100%|██████████| 50/50 [00:08<00:00,  6.00it/s]


In [18]:
track_data[1]

{'danceability': 0.754,
 'energy': 0.583,
 'key': 4,
 'loudness': -7.34,
 'mode': 0,
 'speechiness': 0.0432,
 'acousticness': 0.32,
 'instrumentalness': 0.00015,
 'liveness': 0.144,
 'valence': 0.77,
 'tempo': 122.012,
 'type': 'audio_features',
 'id': '6xZ4Q2k2ompmDppyeESIY8',
 'uri': 'spotify:track:6xZ4Q2k2ompmDppyeESIY8',
 'track_href': 'https://api.spotify.com/v1/tracks/6xZ4Q2k2ompmDppyeESIY8',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6xZ4Q2k2ompmDppyeESIY8',
 'duration_ms': 220051,
 'time_signature': 4}

In [29]:
track_df = pd.json_normalize(track_data)
track_df

danceability  energy  key  loudness  mode  speechiness  acousticness  \
0          0.298  0.3800    6    -8.426     1       0.0287      0.778000   
1          0.754  0.5830    4    -7.340     0       0.0432      0.320000   
2          0.548  0.6080    5    -7.506     1       0.0265      0.004150   
3          0.547  0.7710    3    -4.722     1       0.0284      0.026600   
4          0.514  0.7300    1    -5.934     1       0.0598      0.001460   
5          0.414  0.4760    3    -6.520     1       0.0302      0.910000   
6          0.614  0.6090    5    -5.837     0       0.0411      0.165000   
7          0.570  0.5950    6    -7.833     1       0.0464      0.116000   
8          0.668  0.5740    3   -12.066     1       0.0310      0.036400   
9          0.535  0.0755    4   -20.142     1       0.0313      0.895000   
10         0.592  0.1500    2   -11.669     1       0.1190      0.906000   
11         0.674  0.8650    6    -6.167     1       0.0412      0.046200   
12         0.376  0.2550   11   -18.120     0       0.0361      0.160000   
13         0.561  0.7780    2    -8.608     1       0.0329      0.000022   
14         0.450  0.6020    6    -4.308     1       0.0273      0.089200   
15         0.456  0.2630    0   -10.630     1       0.0332      0.806000   
16         0.553  0.4130    2   -10.037     1       0.0334      0.687000   
17         0.268  0.9500    8    -2.857     1       0.1310      0.016800   
18         0.696  0.4330    4   -14.278     1       0.0296      0.016300   
19         0.220  0.1410    9   -17.272     1       0.0360      0.235000   
20         0.532  0.6230    5    -9.208     1       0.0331      0.538000   
21         0.515  0.4760    9    -9.843     1       0.0328      0.314000   
22         0.436  0.3370    1    -8.680     1       0.0373      0.069800   
23         0.243  0.3300    7   -11.509     1       0.0333      0.744000   
24         0.439  0.3230   10    -6.764     1       0.0323      0.858000   
25         0.641  0.8420    9    -7.270     1       0.0292      0.132000   
26         0.378  0.3540    4   -12.625     1       0.0407      0.848000   
27         0.518  0.9800    5    -1.899     1       0.0480      0.004680   
28         0.735  0.4520    4    -5.628     1       0.0295      0.765000   
29         0.686  0.6620    7    -8.505     1       0.0413      0.467000   
30         0.566  0.6380    9    -7.976     1       0.0294      0.384000   
31         0.777  0.3570    0    -6.942     1       0.0522      0.757000   
32         0.666  0.6900    0    -4.277     0       0.0358      0.012400   
33         0.559  0.3100   10   -10.445     1       0.0338      0.881000   
34         0.596  0.4880    4   -10.769     1       0.0274      0.900000   
35         0.709  0.6550    4    -9.638     1       0.0342      0.542000   
36         0.300  0.4460    0    -9.126     1       0.0340      0.749000   
37         0.649  0.6920    6    -4.909     1       0.0433      0.302000   
38         0.530  0.0940    0   -17.323     1       0.0280      0.975000   
39         0.613  0.5810    0    -8.588     0       0.0424      0.537000   
40         0.518  0.8110    0    -4.381     1       0.0369      0.005480   
41         0.549  0.8070    8    -4.320     1       0.0312      0.006050   
42         0.356  0.8720    2    -7.837     1       0.0680      0.004170   
43         0.669  0.7650    8    -5.535     1       0.0630      0.363000   
44         0.855  0.6760    1    -8.036     1       0.0333      0.214000   
45         0.653  0.4520    4   -11.143     0       0.0550      0.838000   
46         0.645  0.8410    7    -3.727     1       0.1050      0.172000   
47         0.631  0.8370    1    -4.917     1       0.0611      0.008620   
48         0.633  0.7640   11    -7.099     1       0.0336      0.050900   
49         0.610  0.8240    9    -5.217     1       0.0347      0.014000   

    instrumentalness  liveness  valence    tempo            type  \
0           0.000056    0.1100   0.1520   75.602  audio_fea

## Clean Up Track Data

## Update 'mode' to tell if track is major or minor

In [31]:
mod_dict = {0 : 'Minor',
            1: 'Major'}

In [36]:
track_df['mode'].replace(mod_dict, inplace=True)

In [37]:
track_df

danceability  energy  key  loudness   mode  speechiness  acousticness  \
0          0.298  0.3800    6    -8.426  Major       0.0287      0.778000   
1          0.754  0.5830    4    -7.340  Minor       0.0432      0.320000   
2          0.548  0.6080    5    -7.506  Major       0.0265      0.004150   
3          0.547  0.7710    3    -4.722  Major       0.0284      0.026600   
4          0.514  0.7300    1    -5.934  Major       0.0598      0.001460   
5          0.414  0.4760    3    -6.520  Major       0.0302      0.910000   
6          0.614  0.6090    5    -5.837  Minor       0.0411      0.165000   
7          0.570  0.5950    6    -7.833  Major       0.0464      0.116000   
8          0.668  0.5740    3   -12.066  Major       0.0310      0.036400   
9          0.535  0.0755    4   -20.142  Major       0.0313      0.895000   
10         0.592  0.1500    2   -11.669  Major       0.1190      0.906000   
11         0.674  0.8650    6    -6.167  Major       0.0412      0.046200   
12         0.376  0.2550   11   -18.120  Minor       0.0361      0.160000   
13         0.561  0.7780    2    -8.608  Major       0.0329      0.000022   
14         0.450  0.6020    6    -4.308  Major       0.0273      0.089200   
15         0.456  0.2630    0   -10.630  Major       0.0332      0.806000   
16         0.553  0.4130    2   -10.037  Major       0.0334      0.687000   
17         0.268  0.9500    8    -2.857  Major       0.1310      0.016800   
18         0.696  0.4330    4   -14.278  Major       0.0296      0.016300   
19         0.220  0.1410    9   -17.272  Major       0.0360      0.235000   
20         0.532  0.6230    5    -9.208  Major       0.0331      0.538000   
21         0.515  0.4760    9    -9.843  Major       0.0328      0.314000   
22         0.436  0.3370    1    -8.680  Major       0.0373      0.069800   
23         0.243  0.3300    7   -11.509  Major       0.0333      0.744000   
24         0.439  0.3230   10    -6.764  Major       0.0323      0.858000   
25         0.641  0.8420    9    -7.270  Major       0.0292      0.132000   
26         0.378  0.3540    4   -12.625  Major       0.0407      0.848000   
27         0.518  0.9800    5    -1.899  Major       0.0480      0.004680   
28         0.735  0.4520    4    -5.628  Major       0.0295      0.765000   
29         0.686  0.6620    7    -8.505  Major       0.0413      0.467000   
30         0.566  0.6380    9    -7.976  Major       0.0294      0.384000   
31         0.777  0.3570    0    -6.942  Major       0.0522      0.757000   
32         0.666  0.6900    0    -4.277  Minor       0.0358      0.012400   
33         0.559  0.3100   10   -10.445  Major       0.0338      0.881000   
34         0.596  0.4880    4   -10.769  Major       0.0274      0.900000   
35         0.709  0.6550    4    -9.638  Major       0.0342      0.542000   
36         0.300  0.4460    0    -9.126  Major       0.0340      0.749000   
37         0.649  0.6920    6    -4.909  Major       0.0433      0.302000   
38         0.530  0.0940    0   -17.323  Major       0.0280      0.975000   
39         0.613  0.5810    0    -8.588  Minor       0.0424      0.537000   
40         0.518  0.8110    0    -4.381  Major       0.0369      0.005480   
41         0.549  0.8070    8    -4.320  Major       0.0312      0.006050   
42         0.356  0.8720    2    -7.837  Major       0.0680      0.004170   
43         0.669  0.7650    8    -5.535  Major       0.0630      0.363000   
44         0.855  0.6760    1    -8.036  Major       0.0333      0.214000   
45         0.653  0.4520    4   -11.143  Minor       0.0550      0.838000   
46         0.645  0.8410    7    -3.727  Major       0.1050      0.172000   
47         0.631  0.8370    1    -4.917  Major       0.0611      0.008620   
48         0.633  0.7640   11    -7.099  Major       0.0336      0.050900   
49         0.610  0.8240    9    -5.217  Major       0.0347      0.014000   

    instrumentalness  liveness  valence    tempo            type  \
0       

## Update "key" to tell the actually key

In [38]:
music_dic = {
    0: 'C',
    1: 'C#/Db',
    2: 'D',
    3: 'D#/Eb',
    4: 'E',
    5: 'F',
    6: 'F#/Gb',
    7: 'G',
    8: 'G#/Ab',
    9: 'A',
    10: 'A#/Bb',
    11: 'B' 
}

In [39]:
track_df['key'].replace(music_dic, inplace=True)

In [41]:
track_df.head()

danceability  energy    key  loudness   mode  speechiness  acousticness  \
0         0.298   0.380  F#/Gb    -8.426  Major       0.0287       0.77800   
1         0.754   0.583      E    -7.340  Minor       0.0432       0.32000   
2         0.548   0.608      F    -7.506  Major       0.0265       0.00415   
3         0.547   0.771  D#/Eb    -4.722  Major       0.0284       0.02660   
4         0.514   0.730  C#/Db    -5.934  Major       0.0598       0.00146   

   instrumentalness  liveness  valence    tempo            type  \
0          0.000056    0.1100    0.152   75.602  audio_features   
1          0.000150    0.1440    0.770  122.012  audio_features   
2          0.000025    0.0665    0.436   89.997  audio_features   
3          0.000001    0.0819    0.674  122.036  audio_features   
4          0.000095    0.0897    0.334  171.005  audio_features   

                       id                                   uri  \
0  4pvb0WLRcMtbPGmtejJJ6y  spotify:track:4pvb0WLRcMtbPGmtejJJ6y   
1  6xZ4Q2k2ompmDppyeESIY8  spotify:track:6xZ4Q2k2ompmDppyeESIY8   
2  1ci0BoqpvH73L2TJzHhw9y  spotify:track:1ci0BoqpvH73L2TJzHhw9y   
3  4w2tfK0JA8KrVegKnxukf4  spotify:track:4w2tfK0JA8KrVegKnxukf4   
4  0VjIjW4GlUZAMYd2vXMi3b  spotify:track:0VjIjW4GlUZAMYd2vXMi3b   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...   
1  https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...   
2  https://api.spotify.com/v1/tracks/1ci0BoqpvH73...   
3  https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...   
4  https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...   

                                        analysis_url  duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/4pvb...       285634   
1  https://api.spotify.com/v1/audio-analysis/6xZ4...       220051   
2  https://api.spotify.com/v1/audio-analysis/1ci0...       230213   
3  https://api.spotify.com/v1/audio-analysis/4w2t...       235773   
4  https://api.spotify.com/v1/audio-analysis/0VjI...       200040   

   time_signature  
0               4  
1               4  
2               4  
3               4  
4               4

## Add column duration in minutes/seconds

In [47]:
for i in range(len(track_df['id'])):
    millis=track_df['duration_ms'].iloc[i]
    millis = int(millis)
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))%60
    minutes = int(minutes)

    track_df['duration_min'].iloc[i] = "%d:%d" % (minutes, seconds)

C:\Users\Nicholas\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [48]:
track_df.head()

danceability  energy    key  loudness   mode  speechiness  acousticness  \
0         0.298   0.380  F#/Gb    -8.426  Major       0.0287       0.77800   
1         0.754   0.583      E    -7.340  Minor       0.0432       0.32000   
2         0.548   0.608      F    -7.506  Major       0.0265       0.00415   
3         0.547   0.771  D#/Eb    -4.722  Major       0.0284       0.02660   
4         0.514   0.730  C#/Db    -5.934  Major       0.0598       0.00146   

   instrumentalness  liveness  valence    tempo            type  \
0          0.000056    0.1100    0.152   75.602  audio_features   
1          0.000150    0.1440    0.770  122.012  audio_features   
2          0.000025    0.0665    0.436   89.997  audio_features   
3          0.000001    0.0819    0.674  122.036  audio_features   
4          0.000095    0.0897    0.334  171.005  audio_features   

                       id                                   uri  \
0  4pvb0WLRcMtbPGmtejJJ6y  spotify:track:4pvb0WLRcMtbPGmtejJJ6y   
1  6xZ4Q2k2ompmDppyeESIY8  spotify:track:6xZ4Q2k2ompmDppyeESIY8   
2  1ci0BoqpvH73L2TJzHhw9y  spotify:track:1ci0BoqpvH73L2TJzHhw9y   
3  4w2tfK0JA8KrVegKnxukf4  spotify:track:4w2tfK0JA8KrVegKnxukf4   
4  0VjIjW4GlUZAMYd2vXMi3b  spotify:track:0VjIjW4GlUZAMYd2vXMi3b   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...   
1  https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...   
2  https://api.spotify.com/v1/tracks/1ci0BoqpvH73...   
3  https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...   
4  https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...   

                                        analysis_url  duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/4pvb...       285634   
1  https://api.spotify.com/v1/audio-analysis/6xZ4...       220051   
2  https://api.spotify.com/v1/audio-analysis/1ci0...       230213   
3  https://api.spotify.com/v1/audio-analysis/4w2t...       235773   
4  https://api.spotify.com/v1/audio-analysis/0VjI...       200040   

   time_signature duration_min  
0               4         4:45  
1               4         3:40  
2               4         3:50  
3               4         3:55  
4               4         3:20

## Rename track id column

In [51]:
track_df=track_df.rename(columns = {'id':'track_id'})
track_df.head()                     

danceability  energy    key  loudness   mode  speechiness  acousticness  \
0         0.298   0.380  F#/Gb    -8.426  Major       0.0287       0.77800   
1         0.754   0.583      E    -7.340  Minor       0.0432       0.32000   
2         0.548   0.608      F    -7.506  Major       0.0265       0.00415   
3         0.547   0.771  D#/Eb    -4.722  Major       0.0284       0.02660   
4         0.514   0.730  C#/Db    -5.934  Major       0.0598       0.00146   

   instrumentalness  liveness  valence    tempo            type  \
0          0.000056    0.1100    0.152   75.602  audio_features   
1          0.000150    0.1440    0.770  122.012  audio_features   
2          0.000025    0.0665    0.436   89.997  audio_features   
3          0.000001    0.0819    0.674  122.036  audio_features   
4          0.000095    0.0897    0.334  171.005  audio_features   

                 track_id                                   uri  \
0  4pvb0WLRcMtbPGmtejJJ6y  spotify:track:4pvb0WLRcMtbPGmtejJJ6y   
1  6xZ4Q2k2ompmDppyeESIY8  spotify:track:6xZ4Q2k2ompmDppyeESIY8   
2  1ci0BoqpvH73L2TJzHhw9y  spotify:track:1ci0BoqpvH73L2TJzHhw9y   
3  4w2tfK0JA8KrVegKnxukf4  spotify:track:4w2tfK0JA8KrVegKnxukf4   
4  0VjIjW4GlUZAMYd2vXMi3b  spotify:track:0VjIjW4GlUZAMYd2vXMi3b   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...   
1  https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...   
2  https://api.spotify.com/v1/tracks/1ci0BoqpvH73...   
3  https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...   
4  https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...   

                                        analysis_url  duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/4pvb...       285634   
1  https://api.spotify.com/v1/audio-analysis/6xZ4...       220051   
2  https://api.spotify.com/v1/audio-analysis/1ci0...       230213   
3  https://api.spotify.com/v1/audio-analysis/4w2t...       235773   
4  https://api.spotify.com/v1/audio-analysis/0VjI...       200040   

   time_signature duration_min  
0               4         4:45  
1               4         3:40  
2               4         3:50  
3               4         3:55  
4               4         3:20

# Create Master Dataframe

In [53]:
master_df = pd.merge(df2, track_df,
                       how='left', on=['track_id'])

In [54]:
master_df

track_id                              track_name  \
0   4pvb0WLRcMtbPGmtejJJ6y                  exile (feat. Bon Iver)   
1   6xZ4Q2k2ompmDppyeESIY8                        Level of Concern   
2   1ci0BoqpvH73L2TJzHhw9y                        Modern Chemistry   
3   4w2tfK0JA8KrVegKnxukf4                    The Kids Are Alright   
4   0VjIjW4GlUZAMYd2vXMi3b                         Blinding Lights   
5   6xQpOC55lufcqXvSzp7GTb                        Hard To Be Alone   
6   1uddOsj7TyRA13hnS2yDyk              Can't Take My Eyes Off You   
7   2nUV1fiD45RN6cQZ85GDc1                             Salt (Nlmg)   
8   3FRJFImdfX5NSY3QH3jI4u                              Glistening   
9   3flgdcFBWI84DPi4s1jhhd                                Savannah   
10  3iRiQJsQmH0yPIOJn3Y4WQ                           Heart & Arrow   
11  4ZdmTNaBTErD8n9AQE0YaX                           British Bombs   
12  5PNAsZO4JyT8fUzPyKwZ7W                                 Poppies   
13  5vNd5a48igf00CALQgTso1                                Comme Ça   
14  0elCmyfISzkP5tAYTVuYjS                                  August   
15  1MgV7FIyNxIG7WzMRJV5HC                       my tears ricochet   
16  0ZNU020wNYvgW84iljPkPP                              mirrorball   
17  00c805RUf4tZyrlycaA2AQ                            London Girls   
18  1RgmiiAY2FVLuGcG04ah4F                    Something You Needed   
19  2n33zaHpprIqOiSyoC8I5F     Did I Love You (Like I Promised To)   
20  3hUxzQpSfdDqwM3ZTFQY0K                                  august   
21  7kt9e9LFSpN1zQtYEl19o1                       this is me trying   
22  39UQnyET00p3VpvJoEgF5P                    Something In The Air   
23  4kiOoB3NVLKrs61jTNieAr                                Fletcher   
24  7dz48pntblPzJ9mTPiUH81                             At The Door   
25  64lsIF5pw0sJY0gV5kz0RN                 Can I Call You Tonight?   
26  7Cy7wa746ywgkwIOFJtSor    Yoko Ono (Stripped with Matt Maeson)   
27  0klnCzWGjoRQBjoPYe44Gl  Here Comes the End (feat. Judith Hill)   
28  6YjFI4i4mUHL54T67ucGj6                             Missing You   
29  2Eeur20xVqfUoM3Q7EFPFt         the last great american dynasty   
30  2t0TmblCJctqK13OyRdSmD                                    Home   
31  0Jlcvv8IykzHaSmj49uNW8                                   the 1   
32  72z1OAURj2XwHbZdBg3zpV                                Castaway   
33  2NmsngXHeC1GQ9wWrzhOMf                         illicit affairs   
34  6KJqZcs9XDgVck7Lg9QOTC                                   seven   
35  2BIfG6wL1t5wk1KixoK2BV                                 Chicago   
36  7atYf4ccJ8L5QA4dzXcQN1                                Dopamine   
37  0XZV3zE3j10RPG6Zvxsb6n                                  Nickel   
38  38orSZZSkX6gxDC2enAM4k                     It's Ok to Be Human   
39  4R2kfaDFhslZEMJqAFNpdd                                cardigan   
40  5wxjGTx4Q8esdYMd7SWHZI                                  Hollow   
41  0FDtfURDoNGv0sjNnvscPR                                    "99"   
42  5G1sTBGbZT5o4PNRc75RKI                              Lonely Boy   
43  0gbFh1Mx2cTHWVnZp6IypI                              Turn It Up   
44  7qVsMCptRS8tf4eIsQVHYR                     Slam (Angel Miners)   
45  6VsvKPJ4xjVNKpI8VVZ3SV                        invisible string   
46  0xTmJJm3wSlJ1877tW8GrL                    Here Comes the Night   
47  7krfJLNZ3Q1ITV3PKVbyoM                               You And I   
48  21lfQ6cXJF15ubyPLB6qHz                                 Vanilla   
49  554dAjQxsN1DrNL3dKgp5c                    A Toast and a Spirit   

                                       album_name              artist_name  \
0                                        folklore             Taylor Swift   
1                                Level of Concern        Twenty One Pilots   
2                            Tell All Your Friend               Okey Dokey   
3                                             404           Barns Courtney   
4                                     After Hours        

# Create Excel File

In [68]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('PythonExport.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
master_df.to_excel(writer, index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']


for i, col in enumerate(master_df.columns):

    # find length of column i
    column_len = master_df[col].astype(str).str.len().max()
    # Setting the length if the column header is larger
    # than the max column value length
    column_len = max(column_len, len(col)) + 2
    
    # set the column length
    worksheet.set_column(i, i, column_len)

writer.save()